Мета роботи: розробка моделі машинного навчання для автоматичної класифікації текстових повідомлень на предмет наявності
ознак військового досвіду на прикладі повідомлень з Telegram-каналів

Актуальність: завдання є актуальним для OSINT-аналізу та пріоритизації інформаційних потоків в реальному часі 


Методологія навчання: для досягнення високої точності було обрано метод ансамблювання двох моделей, що поєднує:
1) Logistic Regression - лінійна модель для виявлення прямих лексичних залежностей
2) Gradient Boosting - складніша модель, яка сама по собі є ансамблевою, дозволяє знаходити складні, нелінійні патерни

В якості метрики оцінки локально для валідації та на змаганні на платформі Kaggle обрано метрику F1-Score як вдалий
баланс між Precision(точністю) та Recall(повнотою)

In [10]:
import pandas as pd
import re
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import f1_score
from nltk.stem.snowball import SnowballStemmer
from xgboost import XGBClassifier

In [2]:
# Завантаження даних
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('to_answer.csv')

#Заповнення пропусків
train_df['cleaned_message'] = train_df['cleaned_message'].fillna('')
test_df['cleaned_message'] = test_df['cleaned_message'].fillna('')

print(f"Розмір тренувального датасету: {train_df.shape}")
print(f"Розмір тестового датасету: {test_df.shape}")
SPLIT_RANDOM_STATE = 35

Розмір тренувального датасету: (5299, 6)
Розмір тестового датасету: (2271, 7)


В якості препроцесингу використовується спеціальна бібліотека SnowballStemmer для російської мови. Це дозволяє зводити
слова до їх основи(кореня) - це з одного боку зменшує розмірність словника і оптимізує процес навчання, а з іншого - 
покращує узагальнювальну здатність моделі, адже вона краще розуміє логічний сенс та спорідненість слів.
Також застосовується Regex-фільтр(на основі регулярних виразів) для залишення лише кирилииці та латиниці

In [3]:
stemmer = SnowballStemmer("russian")
def stemming_tokenizer(text):
    tokens = re.findall(r'(?u)\b\w\w+\b', text.lower())
    return [stemmer.stem(t) for t in tokens]

X = train_df['cleaned_message']
y = train_df['new_label']

In [4]:
#Ділимо датасет на аргументи та цільову змінну
X = train_df['cleaned_message']
y = train_df['new_label']
#Проводимо поділ датасету на тренувальний та валідаціний набір в співвдошенні 4:1
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=SPLIT_RANDOM_STATE, stratify=y)

Далі використовується векторизація(аспект Feature Engineering) через TF-IDF Vectorizer
(Term Frequency - Inverse Document Frequency). Використовується з параметрами:
1) ngram_range = (1,3) - враховуються не тільки окремі слова, а й фрази з 2-3 слів, що дозволяє частково вловлювати
контекст словосполучень та речень
2) max_features = 50000 - обмеження словника найбільш значущими токенами 

In [5]:
#Проводимо векторизацію
print("Векторизація")
tfidf = TfidfVectorizer(max_features=50000, ngram_range=(1, 3), tokenizer=stemming_tokenizer, token_pattern=None)
X_train_vec = tfidf.fit_transform(X_train)
X_val_vec = tfidf.transform(X_val)

Векторизація


Далі після первинної підготовки даних створюємо ансамбль для навчання, який буде працювати за принципом
голосування(Voting Ensemble). Такий підхід дозволяє компенсувати слабкі сторони однієї моделі сильними
сторонами іншої, зменшуючи загальну дисперсію помилки.
Для нас важливо підібрати оптимальні гіперпараметри для моделей та ансамблю, тому ми спочатку будемо
проводити ручний первинний GridSearch для того, щоб подивитись, яка комбінація параметрів дозволяє 
досягнути найвищого F1-Score на валідації

Перша модель - XgBoost Classifier(Градієнтний бустинг). Градієнтний бустинг використовується для виявлення складних
нелінійних патернів та взаємозв'язків між словами (feature interactions), які не може вловити логістична регресія.
Модель будує ансамбль дерев рішень послідовно, виправляючи помилки попередніх дерев. Параметри:
1) scale_pos_weight=ratio - Cпеціальний параметр для роботи з незбалансованими даними в XGBoost.
    Розрахунок: sum(negative instances) / sum(positive instances).
    Дія: Збільшує вагу градієнта для позитивного класу, змушуючи модель приділяти більше уваги "рідкісним" приклад
    (військовий досвід/загроза).
2) n_estimators = 200 - Кількість дерев в ансамблі. Значення підібрано емпірично як баланс між здатністю до навчання]
та часом тренування.
3) max_depth = 6 - Максимальна глибина одного дерева. Глибина 6 дозволяє моделювати взаємодії між групами слів
(контекст), але запобігає надмірному перенавчанню, яке виникає при глибоких деревах.
4) learning_rate = 0.1 - Швидкість навчання. Стандартне значення, яке забезпечує стабільну збіжність при заданій
кількості дерев
5) n_jobs = -1 - використання всіх доступних ядер процесора
6) eval_metric = "logloss" - метрика оцінки якості під час навчання, використовує Logarithmic Loss - стандартну
функцію втрат для бінарної класифікації.На відміну від простої точності (Accuracy), logloss враховує впевненість
моделі у прогнозі. Модель отримує більший штраф, якщо вона впевнено передбачає неправильний клас. Це змушує алгоритм
калібрувати ймовірності, а не просто вгадувати мітки 0 чи 1.

В даній роботі я не проводжу оптимізацію гіперпараметрів моделі Gradient Boosting, тому дана модель 
створюється і навчається один раз, оскільки немає потреби проганяти її через цикл.

In [6]:
ratio = float(np.sum(y == 0)) / np.sum(y == 1) 
clf_xgb = XGBClassifier(
    scale_pos_weight=ratio,
    n_estimators=200, 
    max_depth=6, 
    learning_rate=0.1,
    random_state=44, 
    n_jobs=-1, 
    eval_metric='logloss'
)
clf_xgb.fit(X_train_vec, y_train)
xgb_preds_val = clf_xgb.predict_proba(X_val_vec)[:, 1]

Друга модель - логістична регресія. Логістична регресія була обрана як базовий алгоритм (baseline), оскільки вона
демонструє високу ефективність на розріджених даних високої розмірності (TF-IDF матриці). Вона відмінно знаходить
лінійні залежності між наявністю певних токенів та цільовим класом.
Параметри:
1) class_weight = "balanced" - Автоматичне балансування ваг класів обернено пропорційно їх частоті. Це критично важливо для максимізації метрики F1, оскільки штраф за помилку на меншості (клас 1) стає вищим.
2) max_iter = 2000 - Збільшена кількість ітерацій солвера. Оскільки матриця ознак дуже велика, стандартних 100
ітерацій часто недостатньо для збіжності градієнтного спуску
3) n_jobs = -1 - використання всіх доступних ядер процесора
4) C = [1,2,3,4,5,6,7,10] - цей параметр змінюється в кожній новій моделі логістичної регресії,
яку ми утворюємо при проході по циклу. Чим більше цей параметр, тим сильніше
моделі дозволено підлаштовуватися під тренувальні дані, що виправдано при великому 
словнику (50k ознак), де важливі рідкісні, але "сильні" слова

Далі ми проводимо ручний Grid Search, де перебираємо параметри коефіцієнта C оберненої регуляризації для 
логістичної регресії(перший зовнішній цикл), ваги моделей в ансамблі(другий зовнішній цикл) та показники
порогу(threshold)(третій цикл, внутрішній). Перебираючи ці комбінації, рахуємо F1-Score на валідаційному
наборі і визначаємо оптимальну комбінацію.

In [11]:
# Перебираємо параметр C для логістичної регресії
c_values = [1.0, 2.0, 3.0, 4.0, 5.0, 7.0, 10.0]
# Перебираємо ваги для ансамблю (LR, XGB)
weight_options = [(1, 1), (1.5, 1), (2, 1), (1, 1.5), (1, 2)]
best_overall_f1 = 0
best_params = {}

In [8]:
for c_val in c_values:
    # Тренуємо LogisticRegression з новим C
    clf_lr_temp = LogisticRegression(
        C=c_val, 
        class_weight='balanced', 
        max_iter=2000, 
        random_state=44, 
        n_jobs=-1
    )
    #Тренуємо модель логістичної регресії з заданими гіперпараметрами
    clf_lr_temp.fit(X_train_vec, y_train)
    #Передбачаємо ймовірності появи обох класів для кожного з семплів і запам'ятовуємо
    #ймовірність появи класу 1
    lr_preds_val = clf_lr_temp.predict_proba(X_val_vec)[:, 1]
    
    # Перебираємо ваги для кожної з двох моделей ансамблю
    for w_lr, w_xgb in weight_options:
        # Ручне ансамблювання ймовірностей (швидше, ніж VotingClassifier)
        ensemble_preds = (lr_preds_val * w_lr + xgb_preds_val * w_xgb) / (w_lr + w_xgb)
        
        # Підбір порогу для цієї комбінації
        current_best_f1 = 0
        current_thresh = 0.5
        
        # Перебір різного рівня порогу(threshold) прийняття рішень
        # для класифікації як клас 1
        for threshold in np.arange(0.2, 0.8, 0.05):
            score = f1_score(y_val, (ensemble_preds >= threshold).astype(int))
            if score > current_best_f1:
                current_best_f1 = score
                current_thresh = threshold
        
        # Якщо ми змогли знайти кращу комбінацію гіперпараметрів(з вищим F1-Score), то 
        # запам'ятовуємо ці параметри
        if current_best_f1 > best_overall_f1:
            best_overall_f1 = current_best_f1
            best_params = {
                'C': c_val,
                'weights': (w_lr, w_xgb),
                'threshold': current_thresh
            }
            print(f"New best! F1: {best_overall_f1:.4f} | C={c_val}, Weights={w_lr}:{w_xgb}, Thresh={current_thresh:.2f}")

print("\n" + "="*30)
print(f"НАЙКРАЩИЙ РЕЗУЛЬТАТ: F1 = {best_overall_f1:.4f}")
print(f"Параметри: {best_params}")
print("="*30)

New best! F1: 0.7903 | C=1.0, Weights=1:1, Thresh=0.55
New best! F1: 0.7921 | C=1.0, Weights=1.5:1, Thresh=0.50
New best! F1: 0.7952 | C=1.0, Weights=2:1, Thresh=0.55
New best! F1: 0.8000 | C=2.0, Weights=1:1, Thresh=0.55
New best! F1: 0.8069 | C=2.0, Weights=1.5:1, Thresh=0.55
New best! F1: 0.8131 | C=3.0, Weights=1.5:1, Thresh=0.55
New best! F1: 0.8139 | C=3.0, Weights=2:1, Thresh=0.55
New best! F1: 0.8166 | C=4.0, Weights=1.5:1, Thresh=0.55
New best! F1: 0.8180 | C=4.0, Weights=2:1, Thresh=0.55
New best! F1: 0.8188 | C=5.0, Weights=2:1, Thresh=0.55
New best! F1: 0.8209 | C=7.0, Weights=1.5:1, Thresh=0.55
New best! F1: 0.8209 | C=10.0, Weights=1.5:1, Thresh=0.50

НАЙКРАЩИЙ РЕЗУЛЬТАТ: F1 = 0.8209
Параметри: {'C': 10.0, 'weights': (1.5, 1), 'threshold': np.float64(0.49999999999999994)}


Після підбору оптимальних гіперпараметрів проводимо фінальне навчання ансамблю методом 
Soft Voting, застосовуючи вже повноцінний VotingClassifier з scikit-learn. 

In [9]:
# 3. ФІНАЛЬНЕ ТРЕНУВАННЯ З НАЙКРАЩИМИ ПАРАМЕТРАМИ
print("Фінальне тренування...")

# Виконуємо TF-IDF векторизацію на всьому датасеті
X_full_vec = tfidf.fit_transform(X)
X_test_vec = tfidf.transform(test_df['cleaned_message'])

# Тренуємо логістичну регресію з оптимальним коефіцієнтом оберненої регуляризації C
final_lr = LogisticRegression(
    C=best_params['C'], 
    class_weight='balanced', 
    max_iter=3000, 
    random_state=44, 
    n_jobs=-1
)
# Тренуємо XGBoost
final_xgb = XGBClassifier(
    n_estimators=200, 
    max_depth=6, 
    learning_rate=0.1,
    scale_pos_weight=ratio, 
    random_state=44, 
    n_jobs=-1, 
    eval_metric='logloss'
)
# Створюємо ансамбль моделей, що працюють за принципом "Soft Voting"
voting_final = VotingClassifier(
    estimators=[('lr', final_lr), ('xgb', final_xgb)],
    voting='soft',
    weights=list(best_params['weights'])
)
#Тренуємо ансамбль
voting_final.fit(X_full_vec, y)
# Перевіряємо прогнози ансамблю і обробляємо їх відповідно до нашого оптимального
# порогу(threshold)
test_proba = voting_final.predict_proba(X_test_vec)[:, 1]
test_predictions = (test_proba >= best_params['threshold']).astype(int)

# Створюємо фінальний submission
submission = pd.DataFrame({
    "row ID": test_df["row ID"],
    "new_label": test_predictions
})
filename = f"submission_tuned_C{best_params['C']}_W{best_params['weights'][0]}-{best_params['weights'][1]}.csv"
submission.to_csv(filename, index=False)
print(f"Файл збережено: {filename}")

Фінальне тренування...
Файл збережено: submission_tuned_C10.0_W1.5-1.csv


Висновки: отже, для даного змагання на Kaggle, було імплеметовано ансамблевий підхід прогнозування, 
застосовано поєднання логістичної регресії та моделі XGBoost. Через пошук по сітці(Grid Search) було
підібрано оптимальну комбінацію гіперпараметрів, яка забезпечила найкраший результат на валідаційному
наборі. Гіперпараметри підбирались для моделі логістичної регресії, а також для балансування важливості
голосів моделей в ансамблі.